# Requirements

In [83]:
!pip install mne
!pip install pandas
!pip install numpy
!pip install mne-icalabel
!pip install os
!pip install scipy
!pip install matplotlib
!pip install torch

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [84]:
# now, import these modules into your code
''' here, we use the wide-spread standard for 
    pandas and numpy library '''
import numpy as np
import pandas as pd
import mne
import os
import scipy
import matplotlib.pyplot as plt
import torch
# you can read more documentation about this
# extension for mne at: 
# https://github.com/mne-tools/mne-icalabel/blob/main/examples/00_iclabel.py
from mne_icalabel import label_components

# Functions



In [13]:
# function to save to a csv
def preprocessed_eeg_to_csv(eeg_data, path):
    # Convert the RawArray data to a pandas DataFrame
    df = pd.DataFrame(data=eeg_data.get_data().T, columns=eeg_data.ch_names);

    path = path.replace("data_porn_addiction\\raw", "data_porn_addiction\\preprocessed");
    # Extract the directory path
    directory_path = os.path.dirname(path);
    # Create the directory if it doesn't exist
    os.makedirs(directory_path, exist_ok=True);
    # Save the DataFrame to a CSV file
    df.to_csv(path, index=False);
    print(f'file: {path} --- has been written')
    return
# files into an organized structure (dictionary)
def get_csv_paths_by_subfolder(root_folder):
    csv_paths_by_subfolder = {};

    # Walk through the directory tree
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            # Check if the file has a .csv extension
            if file.endswith('.csv'):
                # Get the name of the subfolder
                subfolder_name = os.path.basename(root);
                
                # Construct the full path to the CSV file
                csv_path = os.path.join(root, file);

                # Append the CSV path to the subfolder's list in the dictionary
                if subfolder_name not in csv_paths_by_subfolder:
                    csv_paths_by_subfolder[subfolder_name] = [];
                csv_paths_by_subfolder[subfolder_name].append(csv_path);

    return csv_paths_by_subfolder

# function to create eeg_raw objects
def import_preprocessed_signals(path, channels):
    #ch_names = ['P4','O2','P8','T8','C4','Cz','Fz','F4','Fp2','F8','Fp1','F7','F3','C3','T7','P7','P3','O1','Pz'];

    #path to eeg
    eeg_pandas = pd.read_csv(path, usecols=channels, delimiter=",");
    eeg_pandas=eeg_pandas.fillna(0);
    
    return eeg_pandas

# Function to apply bandpass filter to a vector/signal
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs;
    low = lowcut / nyquist;
    high = highcut / nyquist;
    b, a = scipy.signal.butter(order, [low, high], btype='band');
    filtered_data = scipy.signal.filtfilt(b, a, data);
    return filtered_data

# map a filter function to every column of a pd.dataframe
def eeg_apply_filter(data,lowcut, highcut, fs):
    #initialize the df to store the filtered signals
    filtered_signal = pd.DataFrame({});
    for column in data.columns:
        filtered_signal[column] = bandpass_filter(data[column].values, lowcut, highcut, fs);
    return filtered_signal



In [82]:
# channels of interest pertaining to prefrontal and frontal areas.
ch_names = ['Fz','F4','Fp2','F8','Fp1','F7','F3'];

# directory where you stored preprocessed files
preprocessed_folder = "/Users/monica/Documents/data_porn_addiction/prepro"; # Replace this with the path to your root folder

# get all the directories into a {subject: [file1,file2]} structure
csv_paths_by_subject = get_csv_paths_by_subfolder(preprocessed_folder);
#path=csv_paths_by_subject['S1'][0];
test = import_preprocessed_signals(path,ch_names);


ValueError: Invalid file path or buffer object type: <class 'list'>

# Alpha filter

In [15]:
# Define alpha filter parameters
lowcut = 8.0;  # Lower cutoff frequency in Hz
highcut = 12.0;  # Upper cutoff frequency in Hz
fs = 250.0;  # Sampling frequency in Hz
eeg_band = eeg_apply_filter(test,lowcut, highcut, fs);

In [79]:
epoch_length_seconds = 5; overlap_seconds =2.5;
epoch_length_samples = int(epoch_length_seconds * fs)
overlap_samples = int(overlap_seconds * fs)
step_samples = epoch_length_samples - overlap_samples
n_samples = eeg_band.shape[1];

happy = [];
for subject, files in csv_paths_by_subject.items():
     # sepparate emotions
    path = [item for item in files if 'H.csv' in item];
    happy.extend(path);
print(happy)
    
for i in range(0, len(eeg_band) - epoch_length_samples + 1, step_samples):
    pw_absC1 = pow(eeg_band.iloc[0:epoch_length_samples,0],2).mean()
   

['/Users/monica/Documents/data_porn_addiction/prepro/S5/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S2/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S3/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S4/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/s10/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/s11/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S8/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S1/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S6/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S7/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/S9/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/s13/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/s14/H.csv', '/Users/monica/Documents/data_porn_addiction/prepro/s12/H.csv']


In [80]:
type(happy)

list